In [1]:
import cptac
import cptac.utils as ut
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
from scipy import stats
import gseapy as gp
from gseapy.plot import barplot, dotplot

In [2]:
perm_df = pd.read_csv("data/full_10k_permutation.csv")
perm_df

,Cancer,A1BG,A1CF,A2M,A2ML1,A4GALT,AAAS,AACS,AADAC,AADAT,...,ZSWIM9,ZW10,ZWILCH,ZWINT,ZXDA,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
0,ccrcc,0.052810,0.000838,0.029716,NaN,0.337474,0.728086,6.413643e-02,0.350422,0.057700,...,NaN,1.506673e-01,3.202242e-02,1.603670e-01,NaN,0.033585,5.243498e-01,8.419726e-02,3.727352e-01,0.202787
1,en,0.781720,NaN,0.911887,1.389663e-02,0.332572,0.005615,6.073417e-09,NaN,0.309108,...,0.601641,7.465966e-01,7.258794e-06,5.565692e-09,NaN,0.285104,8.265039e-02,5.077769e-01,2.323887e-01,0.196638
2,luad,0.758399,NaN,0.010615,NaN,NaN,0.139531,1.567981e-36,0.022448,0.017033,...,NaN,4.054898e-10,5.038784e-20,1.490761e-05,NaN,0.577845,1.056011e-10,4.521974e-08,2.750287e-15,0.000013
3,hnscc,0.996778,0.776459,0.228337,5.558182e-04,0.040184,0.014557,8.203827e-01,0.000280,NaN,...,NaN,6.634511e-04,1.484058e-01,1.437383e-03,NaN,0.002749,8.339845e-04,8.879081e-01,1.516343e-01,0.762835
4,lscc,0.137994,NaN,0.097044,2.033188e-22,0.154022,0.008081,7.086254e-02,0.456664,NaN,...,0.001427,1.917384e-13,7.815859e-51,2.845191e-33,NaN,0.011741,7.526734e-03,3.401658e-09,1.794113e-06,0.018557


In [3]:
perm_df = perm_df.melt(id_vars='Cancer', var_name = 'Gene', value_name = 'pval')

In [4]:
background_genes = list(pd.unique(perm_df.Gene))

In [5]:
perm_df = perm_df.dropna()

In [6]:
dfs = []
for cancer in pd.unique(perm_df.Cancer):
    df = perm_df[perm_df.Cancer == cancer]
    df['pval'] = df['pval']
    df['significant'] = df['pval'] <= 1 / len(df)
    dfs.append(df)
sig_perm_df = pd.concat(dfs)
sig_perm_df = sig_perm_df[sig_perm_df.significant]
sig_perm_df

<ipython-input-6-553f9cd461d4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pval'] = df['pval']
<ipython-input-6-553f9cd461d4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['significant'] = df['pval'] <= 1 / len(df)


,Cancer,Gene,pval,significant
50,ccrcc,AAGAB,3.184147e-06,True
55,ccrcc,AAK1,3.250300e-10,True
85,ccrcc,AARSD1,9.000008e-06,True
100,ccrcc,AASS,4.782559e-09,True
115,ccrcc,ABAT,2.797660e-48,True
...,...,...,...,...
68304,lscc,ZW10,1.917384e-13,True
68309,lscc,ZWILCH,7.815859e-51,True
68314,lscc,ZWINT,2.845191e-33,True
68334,lscc,ZYX,3.401658e-09,True


In [7]:
sig_genes = pd.DataFrame(sig_perm_df.groupby('Gene').size())
sig_genes = sig_genes[sig_genes[0] > 2]
sig_genes = list(sig_genes.index)
sig_perm_df = sig_perm_df[sig_perm_df.Gene.isin(sig_genes)]
sig_perm_df = sig_perm_df.sort_values('pval')
sig_perm_df = sig_perm_df.reset_index()
ranked_gene_list = list(sig_perm_df.Gene)

In [8]:
from gprofiler import GProfiler
gp = GProfiler(return_dataframe=True)
results_df = gp.profile(organism='hsapiens', query = ranked_gene_list, no_iea=True,
                       ordered=True, no_evidences=False, background= background_genes)

In [9]:
results_df[results_df.source == 'KEGG']

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
134,KEGG,KEGG:03030,DNA replication,2.824389e-07,True,DNA replication,34,143,8,13597,0.055944,0.235294,query_1,[KEGG:00000],"[RFC5, MCM6, PCNA, MCM2, MCM7, MCM5, MCM3, LIG1]","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
189,KEGG,KEGG:04110,Cell cycle,7.009477e-05,True,Cell cycle,113,255,13,13597,0.050980,0.115044,query_1,[KEGG:00000],"[CHEK2, MCM6, PCNA, MCM2, MCM7, CDK1, MCM5, MC...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
203,KEGG,KEGG:01200,Carbon metabolism,1.368736e-04,True,Carbon metabolism,115,1283,29,13597,0.022603,0.252174,query_1,[KEGG:00000],"[GPI, PSAT1, SHMT2, PFKP, PGK1, PKM, HK2, PGAM...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
229,KEGG,KEGG:03430,Mismatch repair,4.034626e-04,True,Mismatch repair,22,62,4,13597,0.064516,0.181818,query_1,[KEGG:00000],"[MSH6, RFC5, PCNA, MSH2]","[[KEGG], [KEGG], [KEGG], [KEGG]]"
235,KEGG,KEGG:01230,Biosynthesis of amino acids,4.498930e-04,True,Biosynthesis of amino acids,74,1283,21,13597,0.016368,0.283784,query_1,[KEGG:00000],"[PYCR1, PSAT1, SHMT2, PFKP, PGK1, ASS1, PKM, A...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
240,KEGG,KEGG:05230,Central carbon metabolism in cancer,6.011252e-04,True,Central carbon metabolism in cancer,66,475,12,13597,0.025263,0.181818,query_1,[KEGG:00000],"[SLC1A5, SLC16A3, PFKP, MET, TP53, LDHA, PKM, ...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
254,KEGG,KEGG:00010,Glycolysis / Gluconeogenesis,1.058171e-03,True,Glycolysis / Gluconeogenesis,63,1283,19,13597,0.014809,0.301587,query_1,[KEGG:00000],"[GPI, PFKP, PGK1, LDHA, PKM, ALDH3A2, HK2, LDH...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
266,KEGG,KEGG:00240,Pyrimidine metabolism,1.820681e-03,True,Pyrimidine metabolism,51,1302,17,13597,0.013057,0.333333,query_1,[KEGG:00000],"[UMPS, CAD, DCTPP1, TK1, RRM1, RRM2, DTYMK, NM...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
281,KEGG,KEGG:00270,Cysteine and methionine metabolism,2.853593e-03,True,Cysteine and methionine metabolism,47,1263,15,13597,0.011876,0.319149,query_1,[KEGG:00000],"[PSAT1, MTAP, AHCYL2, DNMT1, LDHA, IL4I1, SMS,...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
335,KEGG,KEGG:00920,Sulfur metabolism,1.267642e-02,True,Sulfur metabolism,10,1277,6,13597,0.004699,0.600000,query_1,[KEGG:00000],"[PAPSS2, BPNT1, SUOX, TST, MPST, ETHE1]","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG]]"


In [10]:
filtered_results = results_df[results_df.term_size <= 500]
filtered_results = filtered_results[filtered_results.term_size >=15]

In [11]:
filtered_results[filtered_results.source == 'KEGG']

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences
134,KEGG,KEGG:03030,DNA replication,2.824389e-07,True,DNA replication,34,143,8,13597,0.055944,0.235294,query_1,[KEGG:00000],"[RFC5, MCM6, PCNA, MCM2, MCM7, MCM5, MCM3, LIG1]","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
189,KEGG,KEGG:04110,Cell cycle,7.009477e-05,True,Cell cycle,113,255,13,13597,0.050980,0.115044,query_1,[KEGG:00000],"[CHEK2, MCM6, PCNA, MCM2, MCM7, CDK1, MCM5, MC...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
203,KEGG,KEGG:01200,Carbon metabolism,1.368736e-04,True,Carbon metabolism,115,1283,29,13597,0.022603,0.252174,query_1,[KEGG:00000],"[GPI, PSAT1, SHMT2, PFKP, PGK1, PKM, HK2, PGAM...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
229,KEGG,KEGG:03430,Mismatch repair,4.034626e-04,True,Mismatch repair,22,62,4,13597,0.064516,0.181818,query_1,[KEGG:00000],"[MSH6, RFC5, PCNA, MSH2]","[[KEGG], [KEGG], [KEGG], [KEGG]]"
235,KEGG,KEGG:01230,Biosynthesis of amino acids,4.498930e-04,True,Biosynthesis of amino acids,74,1283,21,13597,0.016368,0.283784,query_1,[KEGG:00000],"[PYCR1, PSAT1, SHMT2, PFKP, PGK1, ASS1, PKM, A...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
240,KEGG,KEGG:05230,Central carbon metabolism in cancer,6.011252e-04,True,Central carbon metabolism in cancer,66,475,12,13597,0.025263,0.181818,query_1,[KEGG:00000],"[SLC1A5, SLC16A3, PFKP, MET, TP53, LDHA, PKM, ...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
254,KEGG,KEGG:00010,Glycolysis / Gluconeogenesis,1.058171e-03,True,Glycolysis / Gluconeogenesis,63,1283,19,13597,0.014809,0.301587,query_1,[KEGG:00000],"[GPI, PFKP, PGK1, LDHA, PKM, ALDH3A2, HK2, LDH...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
266,KEGG,KEGG:00240,Pyrimidine metabolism,1.820681e-03,True,Pyrimidine metabolism,51,1302,17,13597,0.013057,0.333333,query_1,[KEGG:00000],"[UMPS, CAD, DCTPP1, TK1, RRM1, RRM2, DTYMK, NM...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
281,KEGG,KEGG:00270,Cysteine and methionine metabolism,2.853593e-03,True,Cysteine and methionine metabolism,47,1263,15,13597,0.011876,0.319149,query_1,[KEGG:00000],"[PSAT1, MTAP, AHCYL2, DNMT1, LDHA, IL4I1, SMS,...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
356,KEGG,KEGG:01240,Biosynthesis of cofactors,1.834084e-02,True,Biosynthesis of cofactors,138,989,25,13597,0.025278,0.181159,query_1,[KEGG:00000],"[UMPS, PSAT1, CAD, FECH, UGDH, IDO1, SHMT2, AK...","[[KEGG], [KEGG], [KEGG], [KEGG], [KEGG], [KEGG..."
